# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


# 1.What test is appropriate for this problem? Does CLT apply?
### we use hypothesis test for comparing two proportions. 

## Conditions for CLT:
### 1. Independence:
           a. Random sample 
           b. n < 10% of the popupaltion
### 2. There should be at least 10 successes and 10 failures in the sample
            a. np >= 10 and n(1-p) >= 10
Each sample should represent a random sample from the population. So, we meet the first condition
Let's check the second condition, i.e, n < 10% of the population

In [5]:
# Seperate resumes by race
df_white=data[data.race=='w']
df_black=data[data.race=='b']

#Number of CV per race:
w_cvs=len(df_white.race)
b_cvs=len(df_black.race)

#Number of calls per race:
w_calls=sum(data[data.race=='w'].call)
b_calls=sum(data[data.race=='b'].call)

# Sample proportions:
w_sample_p = w_calls / w_cvs
b_sample_p = b_calls / b_cvs

w_sample_p,b_sample_p

(0.096509240246406572, 0.064476386036960986)

### As we don't have access to population proportion information we use the pooled proportion: 


$\hat{p}$<sub>Pool</sub> = $\frac{\ Total successes}{\ total  n} $ = $\frac{\ Number successes 1 + Number successes 2}{\ n1+n2} $ = $\frac{\ Number calls w + Number calls b}{\ n1+n2} $



In [6]:
ppool= ((w_calls+b_calls)/(w_cvs+b_cvs))
ppool

0.080492813141683772

### Here n is approximately 8% of the total population. Therefore, we meet the second condition for CLT. 

### Now let's check the third and final condition.
    np >= 10 and n(1-p) >= 10, where p is the probability of success

In [7]:
# Then let's check if n1p_pool >=10, n1(1-p_pool) >=10:

np_w=w_cvs*ppool
n1p_w=w_cvs*(1-ppool)

# and for n2p_pool >=10, n2(1-p_pool) >=10:
np_b=b_cvs*ppool
n1p_b=b_cvs*(1-ppool)

print(np_w,n1p_w, np_b,n1p_b)

196.0 2239.0 196.0 2239.0


#### All these values are greater than 10. Therefore, the third condition is also true.
## So, it met all the conditions for CLT. So, CLT can be applied.


# What are the null and alternate hypotheses?
   * **H<sub>0</sub>**: p<sub>w</sub> = p<sub>b</sub>

   * **H<sub>L</sub>**: p<sub>w</sub> $\neq$ p<sub>b</sub>

# 3.Compute margin of error, confidence interval, and p-value.
* ME = Z<sup>*</sup> · SE<sub>(p<sub>w</sub> - p<sub>b</sub>)</sub>
* Z = (p<sub>w</sub> - p<sub>b</sub>) - Null / SE<sub>(p<sub>w</sub> - p<sub>b</sub>)</sub>

We are going to need t-statistics but before that we are going to calculate the sampling distribution:

$\hat{p}$<sub>diff</sub> = $\hat{p}$<sub>w</sub> - $\hat{p}$<sub>b</sub> $\sim$ N(mean = 0, SE $\hat{p}$<sub>diff</sub>)

where:

SE $\hat{p}$<sub>diff</sub> is equal to: 


In [8]:
# To calculate SE we use the null hypothesis as true. 
# It means that mean = 0, and SE uses p_pool as population proportion:

SE_diff=round((((ppool*(1-ppool))/b_cvs)+((ppool*(1-ppool))/w_cvs))**0.5,2)
SE_diff

0.01

## we need the Z-value to calculate the ME:
Z-value= $\hat{p}$<sub>w</sub> - $\hat{p}$<sub>b</sub> / SE_diff

In [9]:
#z-value
Zvalue= round((w_sample_p - b_sample_p)/SE_diff,2)
print(Zvalue)

# ME 
ME=round(Zvalue*SE_diff,2)
print(ME)


3.2
0.03


Our **MARGINAL ERROR** is z*SE<sub>diff</sub> = 0.03

Let's calculate the **CONFIDENT INTERVAL**:

In [10]:
#  CI = (estimate point + ME, estimate point - ME)

Max_CI= (w_sample_p-b_sample_p)+ME
Min_CI= (w_sample_p-b_sample_p)-ME

print ("The CONFIDENT INTERVAL IS: (",Min_CI, ",", Max_CI,")")

The CONFIDENT INTERVAL IS: ( 0.00203285420945 , 0.0620328542094 )


In [11]:
p = stats.norm.sf(abs(Zvalue))
pvalue = 2*p
pvalue

0.0013742758758316942

# 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
The low value of the p-value indicates that we can reject the Null hypothesis,H0. We have enough evidence to say that race is a factor in callback success. 